# Lab 7
Welcome to Lab 7! This lab covers:
* how to find the best number for a job (optimization),
* how to find the best-fitting line for a scatter plot (regression using optimization), and
* how to compute confidence intervals for properties of regression models (regression inference).

First we'll learn about optimal ice cream truck placement.  Then we'll use regression to get an accurate estimate of the age of the universe from pictures of exploding stars.

In [1]:
# Run this cell, but please don't change it.

# These lines import the Numpy and Datascience modules.
import numpy as np
from datascience import *
from lab_utils import *

# These lines do some fancy plotting magic
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

# These lines load the tests.
from client.api.assignment import load_assignment 
lab07 = load_assignment('longlab07.ok')

# 1. Optimization

Data 8 is branching out into the ice cream market.  We're catering to San Francisco hipsters, so we operate a truck that sells our fresh organic Sriracha-Kale ice cream.  Today we have driven our truck to Ocean Beach, a long, narrow beach on the western coast of the city.

<img src="ocean_beach.jpg">

Upon arriving, we find that our potential customers are spread out along the beach.  We decide we want to park our truck in the location that's closest *on average* to all the customers.  That way, customers will be more likely to come to our truck.

(This may not be a great way to choose our truck's location.  Maybe you can think of a better way to decide on a location.  Computer-based optimization lets you choose whatever criteria you want, within reason!)

We canvas the beach and record the location of each beachgoer in a table called `customers`.  The beach is oriented roughly North/South, and it's narrow, so we ignore how close each beachgoer is to the water, and record only how far north each person is from the southern end of the beach.

<img src="beach_locations.jpg">

Suppose there are 2 people on the beach, at 600 meters and 950 meters from the Southern end, respectively.  If we park our truck at 750 meters, the average distance from our truck to customers is:

$$\frac{|600 - 750| + |950 - 750|}{2},$$

Here's a straightforward way to write that in Python:

In [2]:
(abs(600 - 750) + abs(950 - 750)) / 2

We've packaged this into a function called `average_distance_from_customers` so you can do this calculation with different customers and truck locations.  It takes 2 arguments:
1. an array of customer locations and
2. a truck location.

It returns the average distance of those customers from a truck at that location.  The next cell defines the function and recomputes our distance.

In [3]:
# Given an array of customer locations (numbers) and a potential location
# for our truck (a single number), computes the average distance between
# that truck location and the customers.
def average_distance_from_customers(customer_locations, truck_location):
    return np.mean(np.abs(np.array(customer_locations) - truck_location))

average_distance_from_customers(np.array([600, 950]), 750)

## 1.1. Parking at the average
It seems reasonable to park our truck at the *average* (or "*mean*") location of the customers.  Let's investigate how well that works.

Suppose we have 3 customers, and they're located 800, 1000, and 1800 meters from the Southern end of the beach, respectively.  The mean of those locations is `np.mean([800, 1000, 1800])`, or 1200 meters.

If we park our ice cream truck there, we can compute the average distance from customers like this:

In [4]:
q110_customer_locations = np.array([800, 1000, 1800])
q110_truck_location = np.mean(q110_customer_locations)
q110_avg_distance = average_distance_from_customers(q110_customer_locations, q110_truck_location)
q110_avg_distance

**Question 1.1.1.** If we park our truck at 1100 meters instead, what's the average distance from our truck to customers?  Compute it below by calling the function `average_distance_from_customers` in an appropriate way.

In [5]:
# For convenience, we've named the truck location.
q111_truck_location = 1100

# Fill in q111_avg_distance.  Use code to compute it.
q111_avg_distance = ...
q111_avg_distance

In [6]:
_ = lab07.grade('q111')

Somehow, parking at 1100 meters is better than parking at the mean (1200 meters)!  So the mean can't be the best location.

## 1.2. Finding the best location
Now we'll look at the full dataset and use our computer to find the best truck location.  Run the next cell to load it.  The cell also shows a histogram of the customer locations.

In [7]:
# Just run this cell.
customers = Table.read_table("customers.csv")
customers.hist(bins=np.arange(0, 2001, 100))
customers

Given these customer locations, we want to find a *single location* for our truck.  If we park our truck at that location, we want it to result in the smallest *average distance from our truck to customers*.

First let's write code to evaluate how well any single location meets our goal.  Below we've defined a function called `average_distance`.  It takes a single number as its argument (a truck location) and returns the average distance from that location to the customers in the `customers` table.  (Since we've already written a function to do this for any set of customers, it's pretty easy.)

In [8]:
def average_distance(truck_location):
    return average_distance_from_customers(customers.column(0), truck_location)

average_distance(1150)

Since `average_distance` tells us how well we've achieved our objective, it's called an *objective function*.  So now let's find the distance that produces the smallest value of this objective function.  (If you came up with some better criterion than the average distance to customers, then you would write a different objective function here.  Everything else below would be pretty much the same.)

Here's one way to find the best location:

1. Check a bunch of locations,
2. compute the average distance (the value of our objective function `average_distance`) for each one, and
3. pick the location with the smallest average distance.

This is called a "brute-force" technique because it uses a lot of computation.  But it gets the job done.

To do it manually, it helps to visualize the objective function values for each potential truck location.  The cell below generates such a graph.

In [9]:
# First we'll figure out the locations to try.  We try
# every meter on the beach, plus half a kilometer in
# either direction for good measure.
lowest_location = int(min(customers.column(0))) - 500
highest_location = int(max(customers.column(0))) + 501
locations = np.arange(lowest_location, highest_location, 1)
potential_locations = Table().with_column("Truck distance from south end (m)", locations)

# An array of average distances from customers, one for
# each potential truck location in potential_locations.
average_distances = ...

# Now we make a scatter plot of the average distances with
# potential truck locations on the horizontal axis and the
# average distances from customers on the vertical axis.
locations_with_distances = potential_locations.with_column("Average distance to customers (m)", average_distances)
locations_with_distances.scatter(0, 1)

**Question 1.2.1.** Looking at the graph, what would you say is the best location for our truck, rounded to the nearest 100?  Fill in your guess in the cell below.

In [10]:
# Fill in your guess here:
q121_best_location_guess = ...

In [11]:
_ = lab07.grade('q121')

The built-in function `minimize` does basically the same thing you just did.  It takes as its argument a *function*, the objective function.  It returns an *array* of inputs that produces the smallest output value of the objective function.

Here's how we'd use `minimize` to find the best location for our ice cream truck:

In [12]:
best_location = minimize(average_distance)
best_location

Was your guess close?

# 2. Regression
A *least-squares* regression line for a dataset is the line that produces the smallest possible squared prediction error for those data.  Now that we know how to minimize things, we can do least-squares regression without resorting to formulae!

We're going to use linear regression to estimate the age of the universe.

In the early 20th century, the most popular cosmological theory suggested that the universe had always existed at a fixed size.  Today, the Big Bang theory prevails: Our universe started out very small and is still expanding.

A consequence of this is Hubble's Law, which says that the expansion of the universe creates the appearance that every galaxy near us is moving away from us at a constant rate.  If we extrapolate that motion backwards to the time when everything would have been on top of everything else, that time is (roughly) the beginning of the universe!

(Note: We are not physicists, and this is just a very high-level abstract of the idea.)

## 2.1. Analogy: driving
If you want to dig into this extrapolation idea, here's an analogy.  You can skip this section if you're not interested.

Suppose that some time ago, Mei started driving in a car going 60 miles per hour in a straight line away from us.  We want to know how long she's been driving, but we forgot to record the time when she left.  If we find out that she's 120 miles away, we can infer that she left 2 hours ago.

Run the next cell to see a pictorial interpretation of that logic:

In [13]:
# Just run this cell.
small_driving_example = Table().with_columns([
        "Name", ["Us", "Mei"],
        "Current distance from us (miles)", [0, 120],
        "Speed moving away from us (miles per hour)", [0, 60]
    ])

small_driving_example.scatter(2, 1, s=200, fit_line=True)

# Fancy magic to draw each person's name with their dot.
for i in range(small_driving_example.num_rows):
    name = small_driving_example.column(0).item(i)
    x = small_driving_example.column(2).item(i)
    y = small_driving_example.column(1).item(i)
    plt.scatter([x - 10], [y + 10], s=2000, marker="$\mathrm{" + name + "}$")

The slope of the line is 2 hours.  (The units are vertical-axis units divided by horizontal-axis units, which are $\frac{\texttt{miles}}{\texttt{miles} / \texttt{hour}}$, or hours.)  So that's our answer.

Imagine that you don't know Mei's exact distance or speed, only rough estimates of them.  Then if you drew this line, you'd get a slightly bad estimate of the time since she left.

But suppose you instead measured the distance and speed of *thousands* of people who left you at the same time going different speeds.  If you drew a line between them, the slope of that line would be a pretty good estimate of the time they left, even if the individual measurements weren't exactly right.

The `drivers.csv` dataset contains the speeds and distances-from-start of 100 drivers who all left the same starting location at the same time, driving at a fixed speed on a straight line away from the start.  The measurements aren't exact.  Here we've created a scatter plot and drawn a line through the data.  (We're not using any optimization yet.)

In [14]:
# Just run this cell.
Table.read_table("drivers.csv").scatter(0, 1, fit_line=True)

Eyeballing it, we'd guess the fit line has a slope of around $\frac{17 \text{ miles}}{10 \text{ MPH}}$, or 1.7 hours.  The actual time was an hour and 40 minutes, or 1.667 hours.  We got pretty close.

## 2.2. Back to cosmology
To do the same thing for the universe, we need to know the distance-from-Earth and speed-away-from-Earth of many stars.  Using pictures taken by very accurate telescopes and a lot of physics, astronomers have been able to estimate both.  It turns out that *nearby supernovae* -- stars that have recently died and exploded -- are among the best sources of this data.  This picture taken by the Hubble telescope shows an entire galaxy, with one bright supernova at the bottom left.

<img src="supernova.jpg">

Our astronomical data for today will come from the [Supernova Cosmology Project](http://supernova.lbl.gov/union/) at Lawrence Berkeley Lab.  The original dataset is [here](http://supernova.lbl.gov/union/figures/SCPUnion2.1_mu_vs_z.txt), with (brief) documentation [here](http://supernova.lbl.gov/union/descriptions.html#Magvsz).  Each row in the table corresponds to a supernova near Earth that was observed by astronomers.  From pictures like the one above, the astronomers deduced how far away each supernova was from Earth and how fast it was moving away from Earth.  Their deductions were good, but not perfect.

Run the cell below to load the data into a table called `close_novas` and make a scatter plot with a best-fit line.

In [15]:
# Just run this cell.
close_novas = Table.read_table("close_novas.csv")

close_novas.scatter(1, 0, fit_line=True)
close_novas

Examining the line, it looks like the slope is around 150/.01.  Converting to the right units, we get 15000 million years, or 15 billion.  That sounds about right!

## 2.3. Fitting the line yourself
`fit_line=True` is convenient, but let's peek under the hood.  Inside, `scatter` uses an optimization procedure to compute the line it draws.  The least-squares regression line for our supernova data is
* the line
* with the smallest average (over all the supernovae we observe)
* error,
* squared,
* where the error is the difference between the prediction based on the line and the supernova's actual distance from Earth.

This plot shows one line we could try to use to fit this dataset (*not* the least-squares line), along with the errors made by that line for a few of the supernovas.  Squaring the length of each red bar, then averaging those squared lengths, tells us how badly the line fits.

In [16]:
# Just run this cell.

# Plot all the novas first.
close_novas.scatter("Speed (parsecs/year)", "Distance (million parsecs)")
plt.suptitle("A potential fit line (plus some errors)")

# Over that plot, plot the proposed line, which is a little
# bit too high.
slope = 16000
right_line_end = 0.02
plt.plot([0.0, right_line_end], [0, slope*right_line_end], "b-", linewidth=1)

# Plot some of the errors:
novas_sample = close_novas.take([0, 1, 2, 3, 4, 41])
plt.scatter(novas_sample.column(1), novas_sample.column(0), c="r", zorder=2, s=70)
for i in range(novas_sample.num_rows):
    x = novas_sample.column("Speed (parsecs/year)").item(i)
    y = novas_sample.column("Distance (million parsecs)").item(i)
    line_y = slope*x
    plt.plot([x, x], [y, line_y], "r-")

What we want is to choose a line that minimizes the average squared error.  To simplify things, we'll assume that the vertical intercept of our lines is 0 (since the physical model implies that's true).  So we only have to choose one thing, the *slope* of the line.  We'll do it in steps.

First we need a way to compute errors.  The cell below defines a function to do that.

In [17]:
def predicted_distances(line_slope, speeds):
    return line_slope * speeds

def errors(line_slope):
    predictions = predicted_distances(line_slope, close_novas.column("Speed (parsecs/year)"))
    return predictions - close_novas.column("Distance (million parsecs)") 

For example, we can examine the prediction errors for a line with slope 16,000 million years.

In [18]:
predictions_16000 = predicted_distances(16000, close_novas.column("Speed (parsecs/year)"))
errors_16000 = errors(16000)

plt.scatter(close_novas.column(1), close_novas.column(0), color="black", label="Actual distance")
plt.scatter(close_novas.column(1), predictions_16000, color="blue", label="Predicted distance")
plt.scatter(close_novas.column(1), errors_16000, color="red", label="Prediction error")
plt.ylabel("Distance (million parsecs)")
plt.xlabel("Speed (parsecs/year)")
_ = plt.legend(loc='center left', bbox_to_anchor=(.7, .5))

The predictions are mostly going above the true distances.  If you look at the errors in red, almost all are positive.  That means our line is a little bit too steep.  We'll find a better one.

Now that we can compute errors, we just need to do the "average squared" part.  `mean_squared_error`, defined below, does that.  Given the slope of a line, it computes the mean squared error when we use that line to predict distances for our nova dataset.

In [19]:
def mean_squared_error(slope):
    return np.mean(errors(slope) ** 2)

This is our objective function!  We can graph it to see what slope is best:

In [20]:
slopes = Table().with_column("slope", np.arange(0, 40000, 10))
mses = slopes.with_column("mean squared error", slopes.apply(mean_squared_error, "slope"))
mses.scatter(0, 1)

And, rather than eyeballing the lowest point on the graph, we can compute the best slope by calling `minimize` on our objective function:

In [21]:
best_line_slope = minimize(mean_squared_error)

# This just shows our answer as a nice string, in billions of years.
"{:,} billion years".format(round(best_line_slope/1000, 4))

That slope, as we've seen, is an estimate of the age of the universe.  The current best estimate of the age of the universe is 13.799 billion years.  Did we get close?

# 3. Inference
Our `close_novas` dataset doesn't include all possible close supernovas.  Suppose it was only a random sample from the larger population of all close supernovas.  How much might our estimate have been affected by sampling error?

You might have seen classical techniques for answering this kind of question.  Most statistical packages provide "standard errors" or confidence intervals for the slopes of regression lines.

Instead, we'll use the bootstrap.  We just have to think of our whole procedure -- finding the slope that minimizes the squared error -- as an estimator.  It's analogous to the mean or median.  It's somewhat more complicated to compute, and it involves a table of 2 columns rather than just one array of data.  But it's still just a number we can compute from our data, or from samples of our data.

We'll run that estimator on many resamples from our data, get a different slope each time, and see what those look like.

In [22]:
# Here we've wrapped up the code we wrote before to find the
# least-squares slope for a nova dataset into a function.
def least_squares_slope(novas):
    def mse(potential_slope):
        predictions = predicted_distances(potential_slope, novas.column("Speed (parsecs/year)"))
        prediction_errors = predictions - novas.column("Distance (million parsecs)")
        return np.mean(prediction_errors ** 2)
    return minimize(mse)

def compute_bootstrap_slope():
    resample = close_novas.sample(with_replacement=True)
    return least_squares_slope(resample)

slope_estimates = repeat(compute_bootstrap_slope, 500)
age_estimates = slope_estimates / 1000

## 3.1. Interpreting bootstrap estimates

Now we just have 500 numbers, each one a "potential age estimate" we might have computed if we'd taken another random sample of novas.  It's hard to look at 500 numbers, so we need a visualization or summary.  Here are three ways to understand these numbers.

##### Histogram
Let's just make a histogram of these age estimates:

In [23]:
Table().with_column("age estimate", age_estimates).hist("age estimate", unit="billion years")

##### Line plot
We can also view all the regression *lines* we would have generated.

In [25]:
# Just run this cell.  Read it if you'd like to see how to generate
# plots like these.  It's a little more complicated than the plotting
# we've done; we have to use the lower-level plotting library
# Matplotlib, which is imported as the module named plt.
def plot_nova_lines(num_lines, min_x, max_x):
    # Plot all the actual data.
    close_novas.scatter(1, 0)
    xs = np.array([min_x, max_x])
    some_slopes = Table().with_columns(["index", np.arange(len(slope_estimates)), "slope", slope_estimates]).sample(num_lines)
    
    def plot_line(index, slope):
        predictions = xs*slope
        plt.plot(xs, predictions, lw=1, alpha=4/num_lines, label="line " + str(index))
    
    some_slopes.apply(plot_line, ["index", "slope"])
    plt.xlim([min_x, max_x])
    prediction_lines_message = "%d randomly-selected prediction lines" % num_lines if num_lines < len(slopes) else "all %d prediction lines" % num_lines
    plt.title("Predicted and actual nova distances\n(%s)" % (prediction_lines_message))
    if num_lines < 10:
        plt.legend(loc="center left", bbox_to_anchor=[1.0, 0.5])
    
plot_nova_lines(2, 0, .025)
plot_nova_lines(5, 0, .025)
plot_nova_lines(500, 0, .025)

The last plot has all 500 lines overlaid against each other.  You can see that they overlap very tightly, but not perfectly.

##### Confidence interval
Using the bootstrap slopes, we can generate a 95% confidence interval for the true age of the universe by finding the 2.5th and 97.5th percentiles of this histogram.  In code, that looks like this:

In [ ]:
lower_bound = np.percentile(ags, 2.5)
upper_bound = np.percentile(ages, 97.5)
print("95% confidence interval: [{:0.3f}, {:0.3f}] billion years".format(lower_bound, upper_bound))

Notice that the confidence interval doesn't actually overlap physicists' current best guess at the age of the universe, which is 13.799 billion years.

It's not a matter of having too little data.  If we didn't have as much data, our confidence interval would just be wider to compensate.

Instead, it's because the assumptions behind the method are a little wrong.  Physicists have developed other, more sophisticated models that are more accurate.

A confidence interval only makes a claim about what you'd see if you had observed all the data.  Since the underlying model is a little bit wrong, the method wouldn't give us a perfect answer even if we'd observed all the novas in the universe.